Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Vimarsh Sathia"
let rollno = "CS17B046"

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



val name : string = "Vimarsh Sathia"


val rollno : string = "CS17B046"


## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Lambda Calculus Interpreter

In this assignment, you will implement lambda calculus interpreters that use different reduction strategies. The abstract syntax tree (AST) for lambda expressions is the one that we have seen in class:


```ocaml
type expr = 
  | Var of string
  | Lam of string * expr
  | App of expr * expr
```


You are provided a parser function `parse_string` that converts a string to this AST and a printer function `string_of_expr` that converts the AST to string. For example, 

In [2]:
#use "init.ml"
open Syntax
let parse_string = Lambda_parse.parse_string
let string_of_expr = string_of_expr

let _ = parse_string "(\\x.x) (\\y.y)"
let _ = string_of_expr (App (Var "x",Lam("y",App(Var "y", Var "x"))))

val alpha_equiv : Syntax.expr -> Syntax.expr -> bool = <fun>


val parse_string : string -> Syntax.expr = <fun>


val string_of_expr : Syntax.expr -> string = <fun>


- : Syntax.expr = App (Lam ("x", Var "x"), Lam ("y", Var "y"))


- : string = "x (λy.y x)"


You will need some helper functions to operate over sets. Since we have not studied set data structure in OCaml. We will use lists instead and implement set functionality on top of lists. You can use the functions from the OCaml list standard library for this assignment. The documentation for OCaml list module is available in the [OCaml manual](https://caml.inria.fr/pub/docs/manual-ocaml/libref/List.html). 

## Problem 1

Implement a function

```ocaml
mem : 'a -> 'a list -> bool
```

`mem e l` returns `true` if the element `e` is present in the list. Otherwise, it returns false. 

In [3]:
let rec mem e l = 
    match l with
    |[] -> false
    |h::t-> 
        if h=e then true
        else (mem e t)  
;;

val mem : 'a -> 'a list -> bool = <fun>


In [4]:
assert (mem "b" ["a";"b";"c"] = true);
assert (mem "x" ["a";"b";"c"] = false)

- : unit = ()


## Problem 2

Implement a function

```ocaml
remove : 'a -> 'a list -> 'a list
```

`remove e l` returns a list `l'` with all the element in `l` except `e`. `remove` also preserves the order of the elements not removed. If `e` is not present in `l`, then return `l`.

In [5]:
let rec remove e l = 
  match l with
  |[] -> []
  |h::t -> if h=e then t
           else h::(remove e t) 

val remove : 'a -> 'a list -> 'a list = <fun>


In [6]:
assert (remove "b" ["a";"b";"c"] = ["a";"c"]);
assert (remove "x" ["a";"b";"c"] = ["a";"b";"c"])

- : unit = ()


## Problem 3

Implement a function

```ocaml
union : string list -> string list -> string list
```

`union l1 l2` performs set union of elements in `l1` and `l2`. The elements in the result list `l` must be lexicographically sorted. Hint: You may want to use the functions `List.sort` and `remove_stutter` from assignment 1 to implement union. Here is an example of using `List.sort`. 

In [7]:
assert (List.sort String.compare ["x";"a";"b";"m"] = ["a";"b";"m";"x"])

- : unit = ()


In [8]:
List.sort String.compare ["x";"a";"b";"m"];;

- : String.t list = ["a"; "b"; "m"; "x"]


In [9]:
let rec remove_stutter l =
  match l with 
  |[] -> []
  |curr::next::t ->
    if(curr=next)then remove_stutter ([curr]@t)
    else [curr]@remove_stutter([next]@t)
  |curr::[]->[curr]
;;

let rec union l1 l2 = 
  let a = l1@l2 in
    let b = List.sort String.compare a in
      remove_stutter b
;;

val remove_stutter : 'a list -> 'a list = <fun>


val union : String.t list -> String.t list -> String.t list = <fun>


In [10]:
assert (union ["a"; "c"; "b"] ["d"; "b"; "x"; "a"] = ["a"; "b"; "c"; "d"; "x"])

- : unit = ()


## Problem 4

Implement a function

```ocaml
add : 'a -> 'a list -> 'a list
```

`add e l` does a set addition of element `e` to list `l` and returns a list. The resultant list is sorted. 

In [11]:
let add e l =
  let ans = [e]@l in
    remove_stutter (List.sort String.compare ans)
;;

val add : String.t -> String.t list -> String.t list = <fun>


In [12]:
assert (add "b" ["a";"c"] = ["a";"b";"c"]);
assert (add "a" ["c"; "a"] = ["a";"c"])

- : unit = ()


## Substitution

At the heart of reducing lambda expressions is substitution. Recall from the lecture that substitution requires us to generate fresh variable names that is different from every other name used in the current context. We will use the following helper function to generate fresh names.

In [13]:
let r = ref 0                                                                       
                                                                                    
let fresh s =                                                                       
  let v = !r in                                                                     
  r := !r + 1;                                                                      
  s ^ (string_of_int v)

val r : int ref = {contents = 0}


val fresh : string -> string = <fun>


It uses mutability features of OCaml which we will study in later lectures. You can use the `fresh` function as follows:

In [14]:
let a = fresh "a"
let b = fresh "b"

val a : string = "a0"


val b : string = "b1"


## Problem 5

Implement a function 

```ocaml
free_variables: expr -> string list
```

that returns the free variables in the given lambda term.

In [15]:
let rec free_variables e = 
  match e with
  |Var(x) -> [x]
  |Lam(x,t1) -> remove x (free_variables t1)
  |App(t1,t2) -> union (free_variables t1) (free_variables t2) 

val free_variables : Syntax.expr -> String.t list = <fun>


In [16]:
assert (free_variables (parse_string "\\x.x") = []);
assert (free_variables (parse_string "\\x.y") = ["y"])

- : unit = ()


## Problem 6

Implement the function

```ocaml
substitute : expr -> string -> expr -> expr
```

where `substitute e x v` does `e[v/x]`. For renaming `x` in `Lam(x,y)` with a fresh name, use `Lam (fresh x, ...)`.

In [17]:
let rec substitute expr x s =
  match expr with
  |Var(y) -> if(y=x)then s else Var(y)
  |Lam(y,t1) -> if (y=x) then Lam(y,t1)
                else 
                  if (mem y (free_variables s)) then 
                    let w = fresh y in
                      let tw = substitute t1 y (Var(w)) in
                        Lam(w, substitute tw x s) 
                else Lam(y,substitute t1 x s)
  |App(t1,t2) -> App ((substitute t1 x s) , (substitute t2 x s))

val substitute : Syntax.expr -> string -> Syntax.expr -> Syntax.expr = <fun>


In [18]:
assert (alpha_equiv 
          (substitute (parse_string "\\y.x") "x" (parse_string "\\z.z w")) 
          (parse_string "λy.λz.z w"));
assert (alpha_equiv 
          (substitute (parse_string "\\x.x") "x" (parse_string "y"))
          (parse_string "λx.x"));
assert (alpha_equiv 
          (substitute (parse_string "\\x.y") "y" (parse_string "x"))
          (parse_string "λx0.x"))

- : unit = ()


## Problem 7

Implement a single step of the call-by-value reduction. Implement the function

```ocaml
reduce_cbv : expr -> expr * bool
```

which does a single step of the call-by-value reduction. Recall that call-by-value reduction is determinisitic. Hence, if reduction is possible, then a single rule applies. `reduce e` returns `(e', true)` if reduction is possible and `e'` is the new expression. `reduce e` returns `(e, false)` if reduction is not possible.

In [19]:
let first (e,flag) = e;;
let second (e,flag) = flag;;

let rec reduce_cbv e =
  match e with
  | Var(x)   -> (e,false)
  | Lam(x,m) -> (e,false)
  | App(m,n) -> 
      let m1 = reduce_cbv m in
        let n1 = reduce_cbv n in
          if second m1 then
            (App((first m1),n),true)
          else if second n1 then
            (App(m,(first n1)),true)
          else match m with 
              |Lam(x1,p1) ->((substitute p1 x1 n),true)(*actual reduction step*)
              |Var(x) -> (e,false)
              |App(t1,t2)  -> (e,false)

val first : 'a * 'b -> 'a = <fun>


val second : 'a * 'b -> 'b = <fun>


val reduce_cbv : Syntax.expr -> Syntax.expr * bool = <fun>


In [20]:
let expr, reduced = reduce_cbv (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"));
        
let expr, reduced = reduce_cbv (parse_string "(λx.x) (λz.(λx.x) z)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "λz.(λx.x) z"));
        
let expr, reduced = reduce_cbv (parse_string "λz.(λx.x) z") in
assert (reduced = false &&
        alpha_equiv expr (parse_string "λz.(λx.x) z"));
        
let expr, reduced = reduce_cbv (parse_string "(λx.y) ((λx.x x) (λx.x x))") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λx.y) ((λx.x x) (λx.x x))"));

let expr, reduced = reduce_cbv (parse_string "x y z") in
assert (reduced = false &&
        alpha_equiv expr (parse_string "x y z"))


- : unit = ()


## Problem 8

Implement a single step of the call-by-name reduction. Implement the function

```ocaml
reduce_cbn : expr -> expr option
```

The rest of the instructions are same as `reduce_cbv`.

In [21]:
let rec reduce_cbn e =
  match e with
  |Var(x)   -> (e,false)
  |Lam(x,t) -> (e,false)
  |App(m,n) -> 
              match m with
              | Var(x) -> (e,false)
              | Lam(x1,m1)-> (substitute m1 x1 n,true)
              | App(m1,n1) -> if second (reduce_cbn m) then (App(first (reduce_cbn m),n),true) else (e,false)

val reduce_cbn : Syntax.expr -> Syntax.expr * bool = <fun>


In [22]:
let expr, reduced = reduce_cbn (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"));
        
let expr, reduced = reduce_cbn (parse_string "(λx.x) (λz.(λx.x) z)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "λz.(λx.x) z"));
        
let expr, reduced = reduce_cbn (parse_string "λz.(λx.x) z") in
assert (reduced = false &&
        alpha_equiv expr (parse_string "λz.(λx.x) z"));
        
let expr, reduced = reduce_cbn (parse_string "(λx.y) ((λx.x x) (λx.x x))") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "y"));

let expr, reduced = reduce_cbn (parse_string "(\\x.x x) ((\\z.z) y)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λz.z) y ((λz.z) y)"));

; let expr, reduced = reduce_cbn (parse_string "x y z") in
assert (reduced = false &&
        alpha_equiv expr (parse_string "x y z"))


- : unit = ()


## Problem 9

Implement a single step of the normal order reduction. Implement the function

```ocaml
reduce_normal : expr -> expr * bool
```

The rest of the instructions are same as `reduce_cbv`.

In [23]:
let rec reduce_normal e =
    match e with
      |Var(x)   -> (e,false)
      |Lam(x,t) -> if second (reduce_normal t) then (Lam(x, first (reduce_normal t)),true) else (e,false)
      |App(Lam(x,m),n) -> (substitute m x n,true)
      |App(m,n) ->  if second (reduce_normal m) then (App(first (reduce_normal m),n),true)
                    else if  second (reduce_normal n) then (App(m,first (reduce_normal n)),true)
                    else (e,false)
;;

val reduce_normal : Syntax.expr -> Syntax.expr * bool = <fun>


In [24]:
let expr, reduced = reduce_normal (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"));
        
let expr, reduced = reduce_normal (parse_string "(λx.x) (λz.(λx.x) z)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "λz.(λx.x) z"));
        
let expr, reduced = reduce_normal (parse_string "λz.(λx.x) z") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "λz. z")); 
        
let expr, reduced = reduce_normal (parse_string "(λx.y) ((λx.x x) (λx.x x))") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "y"));

let expr, reduced = reduce_normal (parse_string "(\\x.x x) ((\\z.z) y)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λz.z) y ((λz.z) y)"));
        
let expr, reduced = reduce_normal (parse_string "f (\\x.x x) ((\\z.z) y)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "f (λx.x x) y"));

let expr, reduced = reduce_normal (parse_string "(\\x.(\\z.z) y) (\\x.x x)") in
assert (reduced = true &&
        alpha_equiv expr (parse_string "(λz.z) y"));


- : unit = ()


In [25]:
let rec eval log depth reduce expr =                     
  if depth = 0 then failwith "non-termination?"                                  
  else begin                                                                     
    let expr', reduced = reduce expr in                                           
    if not reduced then expr else begin                                          
      if log then print_endline ("= " ^ (string_of_expr expr'));                 
      eval log (depth-1) reduce expr'                                    
    end                                                                          
  end
  
let eval_cbv = eval true 1000 reduce_cbv
let eval_cbn = eval true 1000 reduce_cbn
let eval_normal = eval true 1000 reduce_normal

val eval :
  bool ->
  int -> (Syntax.expr -> Syntax.expr * bool) -> Syntax.expr -> Syntax.expr =
  <fun>


val eval_cbv : Syntax.expr -> Syntax.expr = <fun>


val eval_cbn : Syntax.expr -> Syntax.expr = <fun>


val eval_normal : Syntax.expr -> Syntax.expr = <fun>


In [26]:
let zero = parse_string "\\f.\\x. x" in
let one = parse_string "\\f.\\x. f x" in
let two = parse_string "\\f.\\x. f (f x)" in
let three = parse_string "\\f.\\x. f (f (f x))" in

let plus = parse_string "λm. λn. λs. λz. m s (n s z)" in
let mult = parse_string "λm. λn. λs. λz. m (n s) z" in

assert (alpha_equiv (eval_normal (App (App (plus, one), two))) three);
print_endline "";
assert (alpha_equiv (eval_normal (App (App (mult, one), three))) three);
print_endline "";
assert (alpha_equiv (eval_normal (App (App (mult, zero), three))) zero)

= (λn.λs.λz.(λf.λx.f x) s (n s z)) (λf.λx.f (f x))
= λs.λz.(λf.λx.f x) s ((λf.λx.f (f x)) s z)
= λs.λz.(λx.s x) ((λf.λx.f (f x)) s z)
= λs.λz.s ((λf.λx.f (f x)) s z)
= λs.λz.s ((λx.s (s x)) z)
= λs.λz.s (s (s z))

= (λn.λs.λz.(λf.λx.f x) (n s) z) (λf.λx.f (f (f x)))
= λs.λz.(λf.λx.f x) ((λf.λx.f (f (f x))) s) z
= λs.λz.(λx.(λf.λx.f (f (f x))) s x) z
= λs.λz.(λf.λx.f (f (f x))) s z
= λs.λz.(λx.s (s (s x))) z
= λs.λz.s (s (s z))

= (λn.λs.λz.(λf.λx.x) (n s) z) (λf.λx.f (f (f x)))
= λs.λz.(λf.λx.x) ((λf.λx.f (f (f x))) s) z
= λs.λz.(λx.x) z
= λs.λz.z


- : unit = ()
